## Netflix EDA 

![](https://i2.wp.com/tvrev.com/wp-content/uploads/2020/02/thibault-penin-AWOl7qqsffM-unsplash-scaled.jpg?fit=2560%2C1709&ssl=1)

In these uniquely challenging times, Netfilx has proven to be a great source of entertainment. The platform has recorded an increase of 23 percent in the number of paid members during the final quarters of 2020 when compared with the same period an year earlier.

Here is an exploratory data analysis done on the 'netflix-shows' dataset.

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame

import missingno

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

sns.set_theme(style="darkgrid")

## Importing Data

In [ ]:
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data.head()

## 1. Data Overview

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
color= ['dimgrey','dimgrey','dimgrey','red','maroon','maroon','dimgrey','dimgrey','dimgrey','dimgrey','dimgrey','dimgrey']
missingno.bar(data,fontsize=10,color=color,figsize=(10,5))
plt.title('MISSING VALUES',fontsize=20)

As we can see, there are a lot of missing values in the director, cast and country columns. I will drop the director column as filling up the values will not be practically possible. Same for the cast column. For the country column, I am going to take the mode of available countries and assign it to the missing values. I will be doing the same for the date added and ratings column.

## MISSING VALUES

In [ ]:
data['country'] = data['country'].fillna(data['country'].mode()[0])
data['date_added'] = data['date_added'].fillna(data['date_added'].mode()[0])
data['rating'] = data['rating'].fillna(data['country'].mode()[0])

In [ ]:
data.drop(['cast','director'],axis='columns',inplace=True)

In [ ]:
data.isna().sum()

Conclusion: All the missing values in the dataset have either been removed or filled. There are no missing values left.

In [ ]:
data.duplicated().sum()

Conclusion: The dataset has 0 duplicated values.

## DATA CLEANING AND MANIPULATION

Adding some new columns:
* Year Added - y_add
* Month Added - m_add
* Princial Country - country_main 
* Target Age Groups - target_age

In [ ]:
data['y_add'] = data['date_added'].apply(lambda x: x.split(" ")[-1])
data['y_add'].head()

In [ ]:
data['m_add'] = data['date_added'].apply(lambda x: x.split(" ")[0])
data['m_add'].head()

In [ ]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}
data['target_age'] = data['rating'].replace(ratings_ages)

In [ ]:
data['country_main'] = data['country'].apply(lambda x: x.split(",")[0])
data['country_main'].head()

Making two new dataframes, one with movies collection and other with TV shows collection:
* movie_df
* tv_df

In [ ]:
movie_df = data[data['type'] == 'Movie']
tv_df = data[data['type'] == 'TV Show']

In [ ]:
movie_df.head()

## Data Visualization

In [ ]:
x = data['type'].value_counts().reset_index()
fig = px.pie(x,values='type',names='index',color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_traces(textposition='inside', textinfo='percent+label',
                  marker = dict(line = dict(color = 'white', width = 1)))

Conclusion: As we can see, the content on Netflix dataset has 30.9% TV shows and 69.1% Movies. 

Now let's look into categorization based upon other factors.

In [ ]:
country_df = data['country_main'].value_counts().reset_index()
country_df = country_df[country_df['country_main'] /  country_df['country_main'].sum() > 0.01]

fig = px.pie(country_df, values='country_main', names='index',color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_traces(textposition='inside', textinfo='percent+label',
                  marker = dict(line = dict(color = 'white', width = 1)))
fig.show()

Conclusion: After dividing the dataset on the basis of country of production, we see that United States holds the highest percentage, of about half of the total content. India comes second in the list with about 14 percentage share.
(Note: We have filled the NaN values in the country_main column with the mean() method. That can be counted as a factor of the dominance of United States)

In [ ]:
def generate_df(df):
    new_df = df.groupby(['y_add', 'm_add']).agg({'show_id': 'count'}).reset_index()
    new_df = new_df[new_df['show_id'] != 0]
    new_df.columns = ['y_add', 'm_add', 'counts']
    new_df = new_df.sort_values('y_add')
    return new_df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

movie_new_df = generate_df(movie_df)
tv_new_df = generate_df(tv_df)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(
    go.Pie(labels=movie_new_df['y_add'], values=movie_new_df['counts']),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=tv_new_df['y_add'], values=tv_new_df['counts']),
    row=1, col=2
)

fig.update_traces(textposition='inside', hole=.4, hoverinfo="label+percent+name",
                  marker = dict(line = dict(color = 'white', width = 2)))

fig.update_layout(
    title_text="Content distribution by Year of Adding",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Movies', x=0.18, y=0.5, font_size=17, showarrow=False, font_color='black'),
                 dict(text='TV Shows', x=0.83, y=0.5, font_size=17, showarrow=False, font_color='black')])
fig.show()

Conclusion: The above plot shows the distribution of Movies and TV Shows separately based upon the year of adding. There is a noticable increase in the number of shows added throughout the years. For movies 2019 records the maximum number while for TV shows the year 2020 records the maximum number of addition of shows. 

In [ ]:
def generate_year_df(df,year):
    new_df = df.groupby(['target_age', 'y_add']).agg({'show_id': 'count'}).reset_index()
    new_df = new_df[new_df['show_id'] != 0]
    new_df.columns = ['target_age', 'y_add', 'counts']
    new_df = new_df.sort_values('y_add')
    new_df= new_df[new_df['y_add']==year]
    return new_df

In [ ]:
group_2012 = generate_year_df(data,'2012')
group_2015 = generate_year_df(data,'2015')
group_2020 = generate_year_df(data,'2020')

fig = make_subplots(1, 3, specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]],
                    subplot_titles=['2012', '2015','2020'])
fig.add_trace(go.Pie(labels=group_2012['target_age'], values=group_2012['counts'],scalegroup='one', 
                     name="Target audience 2012"), 1, 1)

fig.add_trace(go.Pie(labels=group_2015['target_age'], values=group_2015['counts'],scalegroup='one', 
                     name="Target audience 2015"), 1, 2)
fig.add_trace(go.Pie(labels=group_2020['target_age'], values=group_2020['counts'],scalegroup='one', 
                     name="Target audience 2020"), 1, 3)

fig.update_traces(textposition='inside',  hoverinfo="label+percent+name",
                  marker = dict(line = dict(color = 'white', width = 1),
                                colors = ['#3D0C02', '#800000' , '#C11B17','#C0C0C0'],))

fig.update_layout(title_text='Content distribution by target audience 2012 vs 2015 vs 2020')
fig.show()

Conclusion: The above plot shows content distribution by target age. The sizes of te plot is relative to the number of content added in the mentioned years. As it is clear from the plot, there was a huge increase in the number of content from the year 2012 to 2015 to 2020.

In [ ]:
fig = px.histogram(data, x='country_main',color_discrete_sequence=['indianred'])
fig.update_xaxes(categoryorder='total ascending')
fig.show()

Conclusion: The above histogram represents the content added with respect to the country_main column.

In [ ]:
def generate_age_df(df):
    new_df = df.groupby(['target_age']).agg({'show_id': 'count'}).reset_index()
    new_df = new_df[new_df['show_id'] != 0]
    new_df.columns = ['target_age', 'counts']
    new_df = new_df.sort_values('target_age')
    
    return new_df

In [ ]:
movie_age_df = generate_age_df(movie_df)
fig = px.pie(labels = movie_age_df['target_age'], values = movie_age_df['counts'], names = movie_age_df['target_age'], width = 550, height = 550)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(colors = ['#3D0C02', '#800000'  , '#C11B17','#C0C0C0'], 
                                line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Movies', 
                                      x = 0.5, y = 0.5, font_size = 40, showarrow = False,                                       
                                      font_color = 'black')],
                  showlegend = False)

fig.show()

Conclusion: The above plot shows the percentage of movies added based upon the target age. As one can see, the Adults section scores the highest number with 43.8%.

In [ ]:
tv_age_df = generate_age_df(tv_df)
fig = px.pie(labels = tv_age_df['target_age'], values = tv_age_df['counts'], names = tv_age_df['target_age'], width = 550, height = 550)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(colors = ['#3D0C02', '#800000'  , '#C11B17','#C0C0C0'], 
                                line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'TV Series', 
                                      x = 0.5, y = 0.5, font_size = 36, showarrow = False,                                       
                                      font_color = 'black')],
                  showlegend = False)

fig.show()

Conclusion: The above plot shows the percentage of TV series added based upon the target age. As one can see, the Adults section scores the highest number with 42.5%.

In [ ]:
from scipy.stats import norm

sns.distplot(data.loc[data['release_year'] > 2000, 'release_year'],fit= norm, kde=False,
             color='#827839')

Conclusion: The above distplot is based upon the release_year of the movies/shows.

data.head()

In [ ]:
def generate_duration_df(df):
    new_df = df.groupby(['duration']).agg({'show_id': 'count'}).reset_index()
    new_df = new_df[new_df['show_id'] != 0]
    new_df.columns = ['duration', 'counts']
    new_df = new_df.sort_values('duration')
    return new_df

In [ ]:
new_movie_df = generate_duration_df(movie_df)
fig = px.bar(x=new_movie_df['duration'], y = new_movie_df['counts'])
fig.update_traces(marker = dict(color= '#FFA62F'))

fig.update_xaxes(type='category',categoryorder='total ascending')

fig.show()


Conclusion: The above plot is based upon the duration of Movies. As we can see, the duration of 90 minutes records the highest number of movies.

In [ ]:
new_tv_df = generate_duration_df(tv_df)
fig = px.bar(x=new_tv_df['duration'], y = new_tv_df['counts'])
fig.update_traces(marker = dict(color= '#E55451'))
fig.update_xaxes(type='category',categoryorder='total ascending')
fig.show()


Conclusion: The above plot is based upon the seasons of TV series. As we can see, only 1 season records the highest number of Tv series. Discontinuation of shows due to less popularity or pending release of next season might be a contributing factor.|

In [ ]:
def generate_country_df(df):
    new_df = df.groupby(['country_main']).agg({'show_id': 'count'}).reset_index()
    new_df = new_df[new_df['show_id'] != 0]
    new_df.columns = ['country_main', 'counts']
    new_df = new_df.sort_values('country_main')
    return new_df

In [ ]:
tv_country_df = generate_country_df(tv_df)
fig = px.pie(labels = tv_country_df['country_main'], values = tv_country_df['counts'], names = tv_country_df['country_main'], width = 550, height = 550)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'TV Series', 
                                      x = 0.5, y = 0.5, font_size = 40, showarrow = False,                                       
                                      font_color = 'black')],
                  showlegend = False)

fig.show()

Conclusion: Here is another pie plot depicting the percentage of TV series originating from a country. As noticed from the above, United States holds the highest percentage, i.e approx.44%.

In [ ]:
movie_country_df = generate_country_df(movie_df)
fig = px.pie(labels = movie_country_df['country_main'], values = movie_country_df['counts'], names = movie_country_df['country_main'], width = 550, height = 550)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(line = dict(color = 'white', width = 2)))
                                

fig.update_layout(annotations = [dict(text = 'Movies', 
                                      x = 0.5, y = 0.5, font_size = 32, showarrow = False,                                       
                                      font_color = 'black')],
                  showlegend = False)

fig.show()

Conclusion: Here is another pie plot depicting the percentage of Movies originating from a country. As noticed from the above, United States holds the highest percentage, i.e approx.43.3%.

## ENDING NOTE:

This notebook is a compilation of my learnings from various articles/notebooks/tutorials. You might notice a slight resemblace in the way of creating function/plots as they have been inspired by the work of other more experienced people. However, I have tried to change and input my own ideations and variations. Constructive criticism is always welcome.

Attaching some of the inspiring works:
https://www.kaggle.com/dmitryuarov/netflix-eda-with-plotly#Distribution-of-main-genres-in-US,-India-and-Japan